# Rendering
Jack Torrance started working on his new book called Shining. He wants to publish this book online, with the table of contents on the main page and each chapter on a separate page. He sends the first draft of his story:

```
book = {
    'Chapter 1': 'All work and no play makes Jack a dull boy...',
    'Chapter 2': 'All work and no play makes Jack a dull boy...',
    'Chapter 3': 'All work and no play makes Jack a dull boy...',
    'Chapter 4': 'All work and no play makes Jack a dull boy...',
}

```

We created a project named booksite with the book app inside. Then, we created the templates and book directory. Now, we have this tree of our

project:
```
booksite
├── book
│   ├── templates
│   │   └── book
│   ├── urls.py
│   └── views.py
└── booksite
    ├── urls.py
    └── settings.py
```

This novel seems a bit strange, but anyway, let's create an HTML template for the main page with the contents of the book and add it to the book/templates/book/contents.html file:

In [ ]:
<h2> Shining </h2>
<ul>
  {% for chapter in book %}
  <li>
    <a href="/chapter/{{ forloop.counter }}">Chapter {{ forloop.counter }}</a>
  </li>
  {% endfor %}
</ul>

Each unordered list item links to a chapter page, so users can comfortably read each chapter on a separate HTML page.

To return a user to the contents page, we need to implement an HTTP handler with the get method to the ```views.py``` file:

In [ ]:
from django.views import View
from django.shortcuts import render
 
# book variable mentioned above can be declared here
 
class MainPageView(View):
    def get(self, request, *args, **kwargs):
        return render(request, 'book/contents.html', context={'book': book})

We call the render function and pass the template path and the context dictionary with the book to it. All the variables from the context dictionary will be available in the template, and we can access them by key from a dictionary.

There's also another way. You can declare the book variable inside the MainPageView class. In this case, book will be a class field, so when we want to use the variable, we must add self. before.

# TemplateView class
In the example above, we've defined the HTTP handler with the get method. This method is idle, as it delegates all the work to the render function. Is there a more straightforward way to write this in the code?

You can use one of the built-in Django classes — TemplateView. TemplateView renders a template with the context containing parameters captured in the URL. This class can be used as an HTTP handler or inherited to create a new one.

In the following example, we will explore using the TemplateView by creating a single chapter page. First, a new template should be created in the book/templates/book/chapter.html:




In [ ]:
<h2> Chapter {{ n_chapter }} </h2>
<ul>
  {{ content }}
</ul>

In the next step, we are going to make an HTTP handler by creating the new class ChapterView in the ```views.py``` file that inherits from ```TemplateView```. To specify the HTML template path, define the class ```template_name``` property.

In [ ]:
from django.views.generic.base import TemplateView
 
 
class ChapterView(TemplateView):
    template_name = 'book/chapter.html'

The HTTP handler defined in the example above will automatically render the specified HTML template by providing context containing the chapter number defined in the URL. The context works as a dictionary, as in the previous section. However, the content parameter required for rendering the template is not present in the URL path.

To extend the context generated by TemplateView in the background, define the get_context_data method inside the ChapterView class:

In [ ]:
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs) 
        n_chapter = str(context['n_chapter'])
        context['content'] = books['Chapter ' + n_chapter]
        return context

The super().get_context_data() method retrieves the context created by TemplateView. In the next step, the context is extended with the content parameter, and this context will be used for rendering the template.

Defining a correct URL router pattern is the last thing to make the HTTP handler work. ChapterView expects to receive the variable n_chapter as the **kwargs argument of the get_context_data method, so the URL path should contain a path variable named n_chapter of the int type. URL should be placed in the urlpatterns inside the urls.py file:

In [ ]:
path('chapter/<int:n_chapter>', book.views.ChapterView.as_view())

You can see the whole code below:



In [ ]:
from django.views.generic.base import TemplateView
 
class ChapterView(TemplateView):  # create an HTTP handler
    template_name = 'book/chapter.html'  # specify a HTML template
	
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs) 
        # call get_context_data of the parent to add data to the context
        n_chapter = str(context['n_chapter'])
        context['content'] = books['Chapter ' + n_chapter]
        return context  # return the context in which the new data were added

the final tree of the app:

```
booksite
├── book
│   ├── templates
│   │   └── book
│   │       ├── contents.html
│   │       └── chapter.html
│   ├── urls.py
│   └── views.py
└── booksite
    ├── urls.py
    └── settings.py
```